In [1]:
from var_rnn_cells import *
from mvar import *
from data_v2 import *
import pyflux as pf
import numpy as np
import tensorflow_probability as tfp

from scipy.stats import describe, jarque_bera

from tensorflow.keras.layers import GRU, RNN
from matplotlib import pyplot as plt
from tensorflow.keras.layers import InputLayer, Dense, Dropout, Flatten
import tensorflow as tf
import tensorflow.keras.backend as K
import keras

from sklearn.metrics import mean_squared_error

from IPython.display import Audio
sound_file = './code_done.wav'

Using TensorFlow backend.


In [2]:
n_steps = 7 #{15,30,60,90}
# stocks = ['WIKI/XOM', 'WIKI/GS', 'WIKI/MSFT', 'WIKI/JNJ', 'WIKI/ORCL', 'WIKI/NKE', 'WIKI/MCD', 'WIKI/HPQ',
#          'WIKI/JPM', 'WIKI/BAC', 'WIKI/DIS', 'WIKI/WMT', 'WIKI/OXY', 'WIKI/T', 'WIKI/C']
stocks = ['WIKI/GS']

n_stocks = len(stocks)
batch_size = 100
#gamma = [0, 1, 20, 500]
bayes_iterations = 100

n_epochs = 3000


return_period = 'daily' # {"daily", "weekly", "monthly", "quarterly"}

path = './results/' + return_period + '_' + str(n_steps) + '_'

In [3]:
df =np.array(get_data(stocks= stocks, col = return_period))
X,y = split_sequence(df, n_steps)
X_train, X_test, y_train, y_test = train_test(X,y, batch_size= batch_size)


WIKI/GS


In [4]:
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))
    return tf.keras.Sequential([
      tfp.layers.VariableLayer(2 * n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t[..., :n],
                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
          reinterpreted_batch_ndims=1)),
    ])

# Specify the prior over `keras.layers.Dense` `kernel` and `bias`.
def prior_trainable(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
      tfp.layers.VariableLayer(n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t, scale=1),
          reinterpreted_batch_ndims=1)),
    ])

In [5]:
num_batches = X_train.shape[0]/batch_size

kl_loss_weight = 1.0 / num_batches

def log_likelihood(mean, sigma, data):
    s = (data - mean) ** 2 / (2 * (sigma ** 2))
    pdfs = np.exp(- s)
    pdfs /= np.sqrt(2 * np.pi) * sigma
    return np.log(pdfs).sum()

def neg_log_likelihood(y_obs, y_pred):
    dist = tfp.distributions.Normal(loc=y_pred, scale=.01)
    return K.sum(-dist.log_prob(y_obs)) 

## Models: Training, Predicting, and Saving Results

## Feed Forward neural Network

### Deterministic

In [ ]:
model_name = 'ffnn'
estimation = 'deterministic'

K.clear_session()

y_hat_nn_det = np.empty([X_test.shape[0], n_stocks])

for nns in range(n_stocks):

    model = tf.keras.Sequential([
        InputLayer(input_shape = (n_steps, n_stocks)),
        Flatten(),
        Dense(50, activation = 'relu'),
        Dropout(0.3),
        Dense(50, activation = 'relu'),
        Dropout(0.3),
        Dense(50, activation = 'relu'),
        Dropout(0.3),
        Dense(1)
    ])

    model.compile(optimizer= tf.keras.optimizers.Adam(),
                  loss='mse',
                  metrics=['mse'])



    model.fit(X_train, y_train[:, nns], epochs= n_epochs, batch_size= batch_size, validation_split= 0.2, verbose = 0)
    y_hat_nn_det[:,nns] = model.predict(X_test).squeeze()
    mse = mean_squared_error(y_test[:,nns], y_hat_nn_det[:,nns])
    np.savez(str(path + '_' + str(stocks[nns])[5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), y_hat_nn_det[:,nns])
    np.savez(str(path + '_' + str(stocks[nns])[5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)

    K.clear_session()


W1112 01:42:14.968907 25140 deprecation.py:506] From C:\Users\Djordje\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### Bayesian

In [ ]:
import tensorflow_probability as tfp

In [ ]:
num_batches = X_train.shape[0]/batch_size

kl_loss_weight = 1.0 / num_batches


def neg_log_likelihood(y_obs, y_pred):
    dist = tfp.distributions.Normal(loc=y_pred, scale=.01)
    return K.sum(-dist.log_prob(y_obs)) 

In [ ]:
tfd = tfp.distributions

y_hat_nn_bayes = np.empty([X_test.shape[0], n_stocks])

mu_bayes = np.empty([X_test.shape[0], n_stocks])
var_bayes = np.empty([X_test.shape[0], n_stocks])

model_name = 'ffnn'
estimation = 'bayesian'


K.clear_session()

for nns in range(n_stocks):

    model_ffnn_bayes = tf.keras.Sequential([
        InputLayer(input_shape = (n_steps, n_stocks)),
        Flatten(),
        tfp.layers.DenseVariational(50, posterior_mean_field, prior_trainable, kl_weight= kl_loss_weight),
        tfp.layers.DenseVariational(50, posterior_mean_field, prior_trainable, kl_weight= kl_loss_weight),
        tfp.layers.DenseVariational(50, posterior_mean_field, prior_trainable, kl_weight= kl_loss_weight),
        tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight= kl_loss_weight)
    ])

    model_ffnn_bayes.compile(loss = neg_log_likelihood, optimizer= tf.keras.optimizers.Adam(), metrics=['mse'])


    model_ffnn_bayes.fit(X_train, y_train[:, nns], nb_epoch = n_epochs, batch_size= batch_size, validation_split= 0.2, verbose= 1)
    
    mod_pred = np.empty([X_test.shape[0], bayes_iterations])
    
    for biter in range(bayes_iterations):

        mod_pred[:,biter] = (model_ffnn_bayes.predict(X_test)).squeeze()

    mu_bayes[:,nns] = np.mean(mod_pred, axis = 1).squeeze()
    var_bayes[:,nns] = np.var(mod_pred, axis = 1).squeeze()
    
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'var'), var_bayes[:,nns])
    
    mse = mean_squared_error(y_test[:,nns], mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)
    loglik = log_likelihood(mu_bayes[:,nns], var_bayes[:,nns], y_test[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'loglik'), loglik)

    K.clear_session()

W1112 13:40:41.245085 24828 training.py:700] The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 2715 samples, validate on 679 samples
Epoch 1/3000
2715/2715 [==============================] - 0s 117us/sample - loss: 44244531367.8379 - mean_squared_error: 89501.4141 - val_loss: 63171316427.5935 - val_mean_squared_error: 133113.5469
Epoch 2/3000
2715/2715 [==============================] - 0s 21us/sample - loss: 45897585657.3996 - mean_squared_error: 91813.6875 - val_loss: 40816793599.2459 - val_mean_squared_error: 90167.1484
Epoch 3/3000
2715/2715 [==============================] - 0s 20us/sample - loss: 118596032529.6796 - mean_squared_error: 237309.7344 - val_loss: 71486186085.0427 - val_mean_squared_error: 143082.5625
Epoch 4/3000
2715/2715 [==============================] - 0s 22us/sample - loss: 62878701590.6298 - mean_squared_error: 125776.9609 - val_loss: 116597497302.5273 - val_mean_squared_error: 233617.1719
Epoch 5/3000
2715/2715 [==============================] - 0s 23us/sample - loss: 68375296252.6998 - mean_squared_error: 139398.7344 - val_loss: 74985280936.9

2715/2715 [==============================] - 0s 24us/sample - loss: 34772774518.8066 - mean_squared_error: 69828.3438 - val_loss: 73048380387.3461 - val_mean_squared_error: 152886.1250
Epoch 83/3000
2715/2715 [==============================] - 0s 23us/sample - loss: 60966178877.4070 - mean_squared_error: 121953.2500 - val_loss: 25979529982.4919 - val_mean_squared_error: 52165.7969
Epoch 84/3000
2715/2715 [==============================] - 0s 22us/sample - loss: 19065392260.0074 - mean_squared_error: 38356.8516 - val_loss: 10979213381.1841 - val_mean_squared_error: 22078.4082
Epoch 85/3000
2715/2715 [==============================] - 0s 22us/sample - loss: 24254184456.4862 - mean_squared_error: 49457.8203 - val_loss: 26613578755.3932 - val_mean_squared_error: 57165.4922
Epoch 86/3000
2715/2715 [==============================] - 0s 25us/sample - loss: 34104058681.5175 - mean_squared_error: 68392.7656 - val_loss: 24641104731.6171 - val_mean_squared_error: 52695.7422
Epoch 87/3000
2715/271

## LSTM

### Deterministic

In [6]:
from tensorflow.keras.layers import LSTM

model_name = 'lstm'
estimation = 'deterministic'

K.clear_session()

y_hat_lstm_det = np.empty([X_test.shape[0], n_stocks])

for nns in range(n_stocks):

    model_lstm_det = tf.keras.Sequential([
        LSTM(60, activation='tanh', input_shape=(n_steps, n_stocks)),
        Dropout(0.3),
        Dense(1)
    ])

    model_lstm_det.compile(optimizer= tf.keras.optimizers.Adam(),
                  loss='mse',
                  metrics=['mse'])



    model_lstm_det.fit(X_train, y_train[:, nns], epochs= n_epochs, batch_size= batch_size, validation_split= 0.2, verbose = 1)
    y_hat_lstm_det[:,nns] = (model_lstm_det.predict(X_test)).squeeze()
    mse = mean_squared_error(y_test[:,nns], y_hat_lstm_det[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), y_hat_lstm_det[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)

    K.clear_session()


W1112 20:44:39.521364 14188 deprecation.py:506] From C:\Users\Djordje\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W1112 20:44:39.794647 14188 deprecation.py:323] From C:\Users\Djordje\Anaconda3\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2768 samples, validate on 692 samples
Epoch 1/3000
2768/2768 [==============================] - 1s 253us/sample - loss: 6.8583e-04 - mean_squared_error: 6.8583e-04 - val_loss: 2.1304e-04 - val_mean_squared_error: 2.1304e-04
Epoch 2/3000
2768/2768 [==============================] - 0s 79us/sample - loss: 6.7952e-04 - mean_squared_error: 6.7952e-04 - val_loss: 2.3684e-04 - val_mean_squared_error: 2.3684e-04
Epoch 3/3000
2768/2768 [==============================] - 0s 90us/sample - loss: 6.8669e-04 - mean_squared_error: 6.8669e-04 - val_loss: 2.1750e-04 - val_mean_squared_error: 2.1750e-04
Epoch 4/3000
2768/2768 [==============================] - 0s 84us/sample - loss: 6.8242e-04 - mean_squared_error: 6.8242e-04 - val_loss: 2.2435e-04 - val_mean_squared_error: 2.2435e-04
Epoch 5/3000
2768/2768 [==============================] - 0s 83us/sample - loss: 6.7678e-04 - mean_squared_error: 6.7678e-04 - val_loss: 2.1067e-04 - val_mean_squared_error: 2.1067e-04
Epoch 6/3000
2768/2768 [===

Epoch 45/3000
2768/2768 [==============================] - 0s 100us/sample - loss: 6.7175e-04 - mean_squared_error: 6.7175e-04 - val_loss: 2.1573e-04 - val_mean_squared_error: 2.1573e-04
Epoch 46/3000
2768/2768 [==============================] - 0s 125us/sample - loss: 6.7647e-04 - mean_squared_error: 6.7647e-04 - val_loss: 2.1049e-04 - val_mean_squared_error: 2.1049e-04
Epoch 47/3000
2768/2768 [==============================] - 0s 114us/sample - loss: 6.7418e-04 - mean_squared_error: 6.7418e-04 - val_loss: 2.1212e-04 - val_mean_squared_error: 2.1212e-04
Epoch 48/3000
2768/2768 [==============================] - 0s 107us/sample - loss: 6.7527e-04 - mean_squared_error: 6.7527e-04 - val_loss: 2.1922e-04 - val_mean_squared_error: 2.1922e-04
Epoch 49/3000
2768/2768 [==============================] - 0s 85us/sample - loss: 6.7376e-04 - mean_squared_error: 6.7376e-04 - val_loss: 2.1063e-04 - val_mean_squared_error: 2.1063e-04
Epoch 50/3000
2768/2768 [==============================] - 0s 81us

Epoch 89/3000
2768/2768 [==============================] - 0s 104us/sample - loss: 6.7009e-04 - mean_squared_error: 6.7009e-04 - val_loss: 2.1131e-04 - val_mean_squared_error: 2.1131e-04
Epoch 90/3000
2768/2768 [==============================] - 0s 96us/sample - loss: 6.7006e-04 - mean_squared_error: 6.7006e-04 - val_loss: 2.1048e-04 - val_mean_squared_error: 2.1048e-04
Epoch 91/3000
2768/2768 [==============================] - 0s 80us/sample - loss: 6.7315e-04 - mean_squared_error: 6.7315e-04 - val_loss: 2.1078e-04 - val_mean_squared_error: 2.1078e-04
Epoch 92/3000
2768/2768 [==============================] - 1s 196us/sample - loss: 6.7525e-04 - mean_squared_error: 6.7525e-04 - val_loss: 2.2084e-04 - val_mean_squared_error: 2.2084e-04
Epoch 93/3000
2768/2768 [==============================] - 1s 229us/sample - loss: 6.7486e-04 - mean_squared_error: 6.7486e-04 - val_loss: 2.2691e-04 - val_mean_squared_error: 2.2691e-04
Epoch 94/3000
2768/2768 [==============================] - 1s 232us

2768/2768 [==============================] - 0s 83us/sample - loss: 6.7683e-04 - mean_squared_error: 6.7683e-04 - val_loss: 2.1169e-04 - val_mean_squared_error: 2.1169e-04
Epoch 133/3000
2768/2768 [==============================] - 0s 79us/sample - loss: 6.7192e-04 - mean_squared_error: 6.7192e-04 - val_loss: 2.1260e-04 - val_mean_squared_error: 2.1260e-04
Epoch 134/3000
2768/2768 [==============================] - 0s 72us/sample - loss: 6.7347e-04 - mean_squared_error: 6.7347e-04 - val_loss: 2.1944e-04 - val_mean_squared_error: 2.1944e-04
Epoch 135/3000
2768/2768 [==============================] - 1s 247us/sample - loss: 6.7228e-04 - mean_squared_error: 6.7228e-04 - val_loss: 2.1016e-04 - val_mean_squared_error: 2.1016e-04
Epoch 136/3000
2768/2768 [==============================] - 1s 252us/sample - loss: 6.7058e-04 - mean_squared_error: 6.7058e-04 - val_loss: 2.1038e-04 - val_mean_squared_error: 2.1038e-04
Epoch 137/3000
2768/2768 [==============================] - 1s 284us/sample - 

...

2768/2768 [==============================] - 0s 81us/sample - loss: 4.6165e-04 - mean_squared_error: 4.6165e-04 - val_loss: 2.1310e-04 - val_mean_squared_error: 2.1310e-04
Epoch 2350/3000
2768/2768 [==============================] - 0s 83us/sample - loss: 4.6166e-04 - mean_squared_error: 4.6166e-04 - val_loss: 2.1297e-04 - val_mean_squared_error: 2.1297e-04
Epoch 2351/3000
2768/2768 [==============================] - 0s 79us/sample - loss: 4.6649e-04 - mean_squared_error: 4.6649e-04 - val_loss: 2.1912e-04 - val_mean_squared_error: 2.1912e-04
Epoch 2352/3000
2768/2768 [==============================] - 0s 77us/sample - loss: 4.5900e-04 - mean_squared_error: 4.5900e-04 - val_loss: 2.1577e-04 - val_mean_squared_error: 2.1577e-04
Epoch 2353/3000
2768/2768 [==============================] - 0s 79us/sample - loss: 4.9545e-04 - mean_squared_error: 4.9545e-04 - val_loss: 2.1426e-04 - val_mean_squared_error: 2.1426e-04
Epoch 2354/3000
2768/2768 [==============================] - 0s 81us/sample 

Epoch 2393/3000
2768/2768 [==============================] - 0s 82us/sample - loss: 4.5357e-04 - mean_squared_error: 4.5357e-04 - val_loss: 2.2904e-04 - val_mean_squared_error: 2.2904e-04
Epoch 2394/3000
2768/2768 [==============================] - 0s 79us/sample - loss: 4.5537e-04 - mean_squared_error: 4.5537e-04 - val_loss: 2.1681e-04 - val_mean_squared_error: 2.1681e-04
Epoch 2395/3000
2768/2768 [==============================] - 0s 92us/sample - loss: 4.6051e-04 - mean_squared_error: 4.6051e-04 - val_loss: 2.1475e-04 - val_mean_squared_error: 2.1475e-04
Epoch 2396/3000
2768/2768 [==============================] - 0s 86us/sample - loss: 4.5708e-04 - mean_squared_error: 4.5708e-04 - val_loss: 2.1402e-04 - val_mean_squared_error: 2.1402e-04
Epoch 2397/3000
2768/2768 [==============================] - 0s 85us/sample - loss: 4.5314e-04 - mean_squared_error: 4.5314e-04 - val_loss: 2.1495e-04 - val_mean_squared_error: 2.1495e-04
Epoch 2398/3000
2768/2768 [==============================] -

2768/2768 [==============================] - 0s 90us/sample - loss: 4.6071e-04 - mean_squared_error: 4.6071e-04 - val_loss: 2.1359e-04 - val_mean_squared_error: 2.1359e-04
Epoch 2437/3000
2768/2768 [==============================] - 0s 85us/sample - loss: 4.5331e-04 - mean_squared_error: 4.5331e-04 - val_loss: 2.1236e-04 - val_mean_squared_error: 2.1236e-04
Epoch 2438/3000
2768/2768 [==============================] - 0s 81us/sample - loss: 4.5193e-04 - mean_squared_error: 4.5193e-04 - val_loss: 2.1386e-04 - val_mean_squared_error: 2.1386e-04
Epoch 2439/3000
2768/2768 [==============================] - 0s 82us/sample - loss: 4.4915e-04 - mean_squared_error: 4.4915e-04 - val_loss: 2.1377e-04 - val_mean_squared_error: 2.1377e-04
Epoch 2440/3000
2768/2768 [==============================] - 0s 83us/sample - loss: 4.4782e-04 - mean_squared_error: 4.4782e-04 - val_loss: 2.1669e-04 - val_mean_squared_error: 2.1669e-04
Epoch 2441/3000
2768/2768 [==============================] - 0s 83us/sample 

2768/2768 [==============================] - 0s 105us/sample - loss: 4.5096e-04 - mean_squared_error: 4.5096e-04 - val_loss: 2.2066e-04 - val_mean_squared_error: 2.2066e-04
Epoch 2480/3000
2768/2768 [==============================] - 0s 97us/sample - loss: 4.4321e-04 - mean_squared_error: 4.4321e-04 - val_loss: 2.1336e-04 - val_mean_squared_error: 2.1336e-04
Epoch 2481/3000
2768/2768 [==============================] - 0s 82us/sample - loss: 4.4332e-04 - mean_squared_error: 4.4332e-04 - val_loss: 2.1516e-04 - val_mean_squared_error: 2.1516e-04
Epoch 2482/3000
2768/2768 [==============================] - 0s 98us/sample - loss: 4.3843e-04 - mean_squared_error: 4.3843e-04 - val_loss: 2.1790e-04 - val_mean_squared_error: 2.1790e-04
Epoch 2483/3000
2768/2768 [==============================] - 0s 83us/sample - loss: 4.4607e-04 - mean_squared_error: 4.4607e-04 - val_loss: 2.1342e-04 - val_mean_squared_error: 2.1342e-04
Epoch 2484/3000
2768/2768 [==============================] - 0s 106us/sampl

KeyboardInterrupt: 

### Bayesian 

In [ ]:
K.clear_session()

model_name = 'lstm'
estimation = 'bayesian'

mu_bayes = np.empty([X_test.shape[0], n_stocks])
var_bayes = np.empty([X_test.shape[0], n_stocks])

for nns in range(n_stocks):
    model = tf.keras.Sequential([
        RNN(BayesianLSTMCell(num_units = 90, prior = (0,0.01)), unroll = True),
        tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight= kl_loss_weight)
    ])

    model.compile(loss = neg_log_likelihood, optimizer= tf.keras.optimizers.Adam(), metrics=['mse'])

    model.fit(X_train, y_train.squeeze(), nb_epoch = n_epochs, batch_size= batch_size, validation_split= 0.2, verbose=0)
    
    
    mod_pred = np.empty([X_test.shape[0], bayes_iterations])

    for biter in range(bayes_iterations):

        mod_pred[:,biter] = (model.predict(X_test)).squeeze()

    mu_bayes[:,nns] = np.mean(mod_pred, axis = 1)
    var_bayes[:,nns] = np.var(mod_pred, axis = 1)
    
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'var'), var_bayes[:,nns])
    
    mse = mean_squared_error(y_test[:,nns], mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)
    loglik = log_likelihood(mu_bayes[:,nns], var_bayes[:,nns], y_test[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'loglik'), loglik)

    K.clear_session()

W1112 03:31:52.967802 25140 deprecation.py:323] From c:\master\thesis\code_v2\tfcell_notmir\var_rnn_cells.py:52: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W1112 03:31:52.972301 25140 training.py:700] The `nb_epoch` argument in `fit` has been renamed `epochs`.
W1112 03:31:52.976291 25140 deprecation.py:323] From C:\Users\Djordje\Anaconda3\lib\site-packages\tensorflow_core\python\ops\rnn_cell_impl.py:735: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
W1112 03:31:52.987260 25140 deprecation.py:506] From C:\Users\Djordje\Anaconda3\lib\site-packages\tensorflow_core\python\ops\rnn_cell_impl.py:739: calling Zeros.__init__ (from tensorflow.py

## GRU

### Deterministic

In [ ]:
from tensorflow.keras.layers import GRU

K.clear_session()

model_name = 'gru'
estimation = 'deterministic'

y_hat_gru_det = np.empty([X_test.shape[0], n_stocks])

for nns in range(n_stocks):

    model_gru_det = tf.keras.Sequential([
        GRU(60, activation='tanh', input_shape=(n_steps, n_stocks)),
        Dropout(0.3),
        Dense(1)
    ])

    model_gru_det.compile(optimizer= tf.keras.optimizers.Adam(),
                  loss='mse',
                  metrics=['mse'])



    model_gru_det.fit(X_train, y_train[:, nns], epochs= n_epochs, batch_size= batch_size, validation_split= 0.2, verbose = 0)
    y_hat_gru_det[:,nns] = (model_gru_det.predict(X_test)).squeeze()
    
    mse = mean_squared_error(y_test[:,nns], y_hat_gru_det[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), y_hat_gru_det[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)

    K.clear_session()


### Bayesian

In [ ]:
from var_rnn_cells import *

model_name = 'gru'
estimation = 'bayesian'

mu_bayes = np.empty([X_test.shape[0], n_stocks])
var_bayes = np.empty([X_test.shape[0], n_stocks])

K.clear_session()

for nns in range(n_stocks):
    model = tf.keras.Sequential([
        RNN(BayesianGRUCell(num_units = 90, prior = (0,0.01)), unroll = True),
        tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight= kl_loss_weight)
    ])

    model.compile(loss = neg_log_likelihood, optimizer= tf.keras.optimizers.Adam(), metrics=['mse'])

    model.fit(X_train, y_train.squeeze(), nb_epoch = n_epochs, batch_size= batch_size, validation_split= 0.2, verbose=0)
    
    
    mod_pred = np.empty([X_test.shape[0], bayes_iterations])

    for biter in range(bayes_iterations):

        mod_pred[:,biter] = (model.predict(X_test)).squeeze()

    mu_bayes[:,nns] = np.mean(mod_pred, axis = 1)
    var_bayes[:,nns] = np.var(mod_pred, axis = 1)
    
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'var'), var_bayes[:,nns])
    
    mse = mean_squared_error(y_test[:,nns], mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)
    loglik = log_likelihood(mu_bayes[:,nns], var_bayes[:,nns], y_test[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'loglik'), loglik)

    K.clear_session()

W1112 07:28:54.690885 25140 deprecation.py:323] From c:\master\thesis\code_v2\tfcell_notmir\var_rnn_cells.py:196: GRUCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
W1112 07:28:54.693878 25140 training.py:700] The `nb_epoch` argument in `fit` has been renamed `epochs`.
W1112 07:28:54.706843 25140 deprecation.py:506] From C:\Users\Djordje\Anaconda3\lib\site-packages\tensorflow_core\python\ops\rnn_cell_impl.py:565: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## tanh RNN

### Deterministic

In [ ]:
from tensorflow.keras.layers import SimpleRNNCell

K.clear_session()

model_name = 'tanh'
estimation = 'deterministic'

y_hat_tanh_det = np.empty([X_test.shape[0], n_stocks])

for nns in range(n_stocks):

    model_tanh_det = tf.keras.Sequential([
        RNN(SimpleRNNCell(60, activation='tanh', input_shape=(n_steps, n_stocks))),
        Dropout(0.3),
        Dense(1)
    ])

    model_tanh_det.compile(optimizer= tf.keras.optimizers.Adam(),
                  loss='mse',
                  metrics=['mse'])



    model_tanh_det.fit(X_train, y_train[:, nns], epochs= n_epochs, batch_size= batch_size, validation_split= 0.2, verbose = 0)
    y_hat_tanh_det[:,nns] = (model_tanh_det.predict(X_test)).squeeze()
    
    mse = mean_squared_error(y_test[:,nns], y_hat_tanh_det[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), y_hat_tanh_det[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)

    K.clear_session()


### Bayesian

In [ ]:
K.clear_session()

model = 'tanh'
estimation = 'bayesian'

mu_bayes = np.empty([X_test.shape[0], n_stocks])
var_bayes = np.empty([X_test.shape[0], n_stocks])

for nns in range(n_stocks):
    model = tf.keras.Sequential([
        RNN(BayesianRNNtanh(num_units = 90, prior = (0,0.01)), unroll = True),
        tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight= kl_loss_weight)
    ])

    model.compile(loss = neg_log_likelihood, optimizer= tf.keras.optimizers.Adam(), metrics=['mse'])

    model.fit(X_train, y_train.squeeze(), nb_epoch = n_epochs, batch_size= batch_size, validation_split= 0.2, verbose=0)
    
    
    mod_pred = np.empty([X_test.shape[0], bayes_iterations])

    for biter in range(bayes_iterations):

        mod_pred[:,biter] = (model.predict(X_test)).squeeze()

    mu_bayes[:,nns] = np.mean(mod_pred, axis = 1)
    var_bayes[:,nns] = np.var(mod_pred, axis = 1)
    
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'var'), var_bayes[:,nns])
    
    mse = mean_squared_error(y_test[:,nns], mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)
    loglik = log_likelihood(mu_bayes[:,nns], var_bayes[:,nns], y_test[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'loglik'), loglik)

    K.clear_session()

## reLu RNN

### Deterministic

In [ ]:
K.clear_session()

model_name = 'relu'
estimation = 'deterministic'

y_hat_relu_det = np.empty([X_test.shape[0], n_stocks])

for nns in range(n_stocks):

    model_relu_det = tf.keras.Sequential([
        RNN(SimpleRNNCell(60, activation='relu', input_shape=(n_steps, n_stocks))),
        Dropout(0.3),
        Dense(1)
    ])

    model_relu_det.compile(optimizer= tf.keras.optimizers.Adam(),
                  loss='mse',
                  metrics=['mse'])



    model_relu_det.fit(X_train, y_train[:, nns], epochs= n_epochs, batch_size= batch_size,validation_split= 0.2, verbose = 0)
    y_hat_relu_det[:,nns] = (model_relu_det.predict(X_test)).squeeze()
    
    mse = mean_squared_error(y_test[:,nns], y_hat_relu_det[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), y_hat_relu_det[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)

    K.clear_session()


### Bayesian

In [ ]:
K.clear_session()

model_name = 'relu'
estimation = 'bayesian'

for nns in range(n_stocks):
    model = tf.keras.Sequential([
        RNN(BayesianRNNrelu(num_units = 90, prior = (0,0.01)), unroll = True),
        tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight= kl_loss_weight)
    ])

    model.compile(loss = neg_log_likelihood, optimizer= tf.keras.optimizers.Adam(), metrics=['mse'])

    model.fit(X_train, y_train.squeeze(), nb_epoch = n_epochs, batch_size= batch_size, validation_split= 0.2, verbose=0)
    
    
    mod_pred = np.empty([X_test.shape[0], bayes_iterations])

    for biter in range(bayes_iterations):

        mod_pred[:,biter] = (model.predict(X_test)).squeeze()

    mu_bayes[:,nns] = np.mean(mod_pred, axis = 1)
    var_bayes[:,nns] = np.var(mod_pred, axis = 1)
    
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'var'), var_bayes[:,nns])
    
    mse = mean_squared_error(y_test[:,nns], mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)
    loglik = log_likelihood(mu_bayes[:,nns], var_bayes[:,nns], y_test[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'loglik'), loglik)

    K.clear_session()

## Vector Autoregressive Model

### Deterministic

In [ ]:
model_name = 'var'
estimation = 'deterministic'

K.clear_session()

y_hat_nn_det = np.empty([X_test.shape[0], n_stocks])

for nns in range(n_stocks):

    model = tf.keras.Sequential([
        InputLayer(input_shape = (n_steps, n_stocks)),
        Flatten(),
        Dense(1)
    ])

    model.compile(optimizer= tf.keras.optimizers.Adam(),
                  loss='mse',
                  metrics=['mse'])



    model.fit(X_train, y_train[:, nns], epochs= n_epochs, batch_size= batch_size, verbose = 0)
    y_hat_nn_det[:,nns] = model.predict(X_test).squeeze()
    mse = mean_squared_error(y_test[:,nns], y_hat_nn_det[:,nns])
    np.savez(str(path + '_' + str(stocks[nns])[5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), y_hat_nn_det[:,nns])
    np.savez(str(path + '_' + str(stocks[nns])[5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)

    K.clear_session()


### Bayesian

In [ ]:
tfd = tfp.distributions

#y_hat_var_bayes = np.empty([X_test.shape[0], n_stocks])

mu_bayes = np.empty([X_test.shape[0], n_stocks])
var_bayes = np.empty([X_test.shape[0], n_stocks])

model_name = 'var'
estimation = 'bayesian'


K.clear_session()

for nns in range(n_stocks):

    model_ffnn_bayes = tf.keras.Sequential([
        InputLayer(input_shape = (n_steps, n_stocks)),
        Flatten(),
        tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight= kl_loss_weight)
    ])

    model_ffnn_bayes.compile(loss = neg_log_likelihood, optimizer= tf.keras.optimizers.Adam(), metrics=['mse'])


    model_ffnn_bayes.fit(X_train, y_train[:, nns], nb_epoch = n_epochs, batch_size= batch_size, validation_split= 0.2, verbose= 0)
    
    mod_pred = np.empty([X_test.shape[0], bayes_iterations])
    
    for biter in range(bayes_iterations):

        mod_pred[:,biter] = (model_ffnn_bayes.predict(X_test)).squeeze()

    mu_bayes[:,nns] = np.mean(mod_pred, axis = 1).squeeze()
    var_bayes[:,nns] = np.var(mod_pred, axis = 1).squeeze()
    
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mu'), mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'var'), var_bayes[:,nns])
    
    mse = mean_squared_error(y_test[:,nns], mu_bayes[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'mse'), mse)
    loglik = log_likelihood(mu_bayes[:,nns], var_bayes[:,nns], y_test[:,nns])
    np.savez(str(path + '_' + stocks[nns][5:] + '_' + model_name + '_' + estimation + '_' + 'loglik'), loglik)

    K.clear_session()

In [ ]:
Audio(sound_file, autoplay=True)

In [ ]:
import datetime
print(datetime.datetime.now())